In [3]:
import torch

In [4]:
import os
import numpy as np
print(os.getcwd())

/home/yutingd/LatentSB


In [5]:
rest_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/"
task_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/"


In [6]:
%cd /nfs/turbo/sph-jianghui1/yutingd/HCP/

/nfs/turbo/sph-jianghui1/yutingd/HCP


In [7]:
rest_ids = {
    f.split("_")[0]
    for f in os.listdir(rest_dir)
    if f.endswith("_fALFF.nii")
}

task_ids = {
    f.split("_")[0]
    for f in os.listdir(task_dir)
    if f.endswith("_2bk-0bk.nii")
}

# Intersection = matched subjects in BOTH folders
matched_ids = sorted(list(rest_ids.intersection(task_ids)))

print(f"Found {len(matched_ids)} matched subjects.")
print("Example IDs:", matched_ids[:10])

# -------------------------
# 3. Train / Val / Test split (7 : 2 : 1)
# -------------------------
np.random.seed(42)
matched_ids = np.array(matched_ids)
np.random.shuffle(matched_ids)

n = len(matched_ids)
n_train = int(0.7 * n)
n_val   = int(0.2 * n)
n_test  = n - n_train - n_val

training_ids   = matched_ids[:n_train]
validation_ids = matched_ids[n_train : n_train + n_val]
test_ids       = matched_ids[n_train + n_val :]

print(f"Train: {len(training_ids)}, Val: {len(validation_ids)}, Test: {len(test_ids)}")

# -------------------------
# 4. Write to files
# -------------------------
with open("training_ids.txt", "w") as f:
    for sid in training_ids:
        f.write(f"{sid}\n")

with open("validation_ids.txt", "w") as f:
    for sid in validation_ids:
        f.write(f"{sid}\n")

with open("test_ids.txt", "w") as f:
    for sid in test_ids:
        f.write(f"{sid}\n")

print("Files written:")
print(" - training_ids.txt")
print(" - validation_ids.txt")
print(" - test_ids.txt")

Found 714 matched subjects.
Example IDs: ['100206', '100307', '100408', '100610', '101006', '101107', '101309', '102311', '102513', '102715']
Train: 499, Val: 142, Test: 73
Files written:
 - training_ids.txt
 - validation_ids.txt
 - test_ids.txt


In [8]:
def load_ids(path):
    with open(path, "r") as f:
        return [line.strip() for line in f.readlines()]

train_ids = load_ids("training_ids.txt")
val_ids   = load_ids("validation_ids.txt")
test_ids  = load_ids("test_ids.txt")

print(len(train_ids), len(val_ids), len(test_ids))


499 142 73


In [9]:
%cd /home/yutingd/LatentSB/

/home/yutingd/LatentSB


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [11]:
print(os.getcwd())

/home/yutingd/LatentSB


In [10]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
###############################################################
# train_autoencoder.py — full working script
###############################################################

import torch
import torch.nn.functional as F
from models.ae_3d import Encoder3D, Decoder3D

device = "cuda" if torch.cuda.is_available() else "cpu"

encoder = Encoder3D(latent_dim=128).to(device)
decoder = Decoder3D(latent_dim=128).to(device)

optimizer = torch.optim.Adam(
    list(encoder.parameters()) + list(decoder.parameters()),
    lr=3e-5
)

def train_autoencoder(epochs=10):
    for epoch in range(epochs):
        encoder.train()
        decoder.train()
        total_loss = 0.0

        for rest, task in train_loader:
            # combine rest + task so AE learns shared latent space
            X = torch.cat([rest, task], dim=0).to(device)   # (2, 1, H, W, D)

            z, feat_shape = encoder(X)
            X_hat = decoder(z, feat_shape, X.shape)
            loss = F.mse_loss(X_hat, X)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}: loss = {total_loss/len(train_loader):.4f}")

# run it
train_autoencoder(epochs=5)


In [13]:
rest_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/"
task_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/"

mask_path = None#"/nfs/turbo/sph-jianghui1/yutingd/HCP/AAL_MNI_2mm.nii" #None  # or "/path/to/brain_mask.nii.gz"

    #train_ids = load_ids("training_ids.txt")
    #val_ids = load_ids("validation_ids.txt")

device = "cuda" if torch.cuda.is_available() else "cpu"
latent_dim = 128
batch_size = 1
epochs = 50
lr = 1e-4

    # ---- datasets & loaders ----
train_ds = HCPRestTaskDataset(
    rest_dir=rest_dir,
    task_dir=task_dir,
    subject_ids=train_ids,
    mode="3d",
    mask_path=mask_path,
    normalize=None,
)
val_ds = HCPRestTaskDataset(
    rest_dir=rest_dir,
    task_dir=task_dir,
    subject_ids=val_ids,
    mode="3d",
    mask_path=mask_path,
    normalize=None,
)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    

TypeError: HCPRestTaskDataset.__init__() got an unexpected keyword argument 'mode'

In [17]:
train_ds.

In [13]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler



In [ ]:
rest_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/"
task_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/"

mask_path = "/nfs/turbo/sph-jianghui1/yutingd/HCP/AAL_MNI_2mm.nii" #None  # or "/path/to/brain_mask.nii.gz"

    #train_ids = load_ids("training_ids.txt")
    #val_ids = load_ids("validation_ids.txt")

device = "cuda" if torch.cuda.is_available() else "cpu"
latent_dim = 128
batch_size = 1
epochs = 50
lr = 1e-4

    # ---- datasets & loaders ----
train_ds = HCPRestTaskDataset(
        rest_dir=rest_dir,
        task_dir=task_dir,
        subject_ids=train_ids,
        mode="3d",
        mask_path=mask_path,
        normalize=None,
    )
val_ds = HCPRestTaskDataset(
        rest_dir=rest_dir,
        task_dir=task_dir,
        subject_ids=val_ids,
        mode="3d",
        mask_path=mask_path,
        normalize=None,
    )

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    # ---- models ----
encoder = Encoder3D(latent_dim=latent_dim).to(device)
decoder = Decoder3D(latent_dim=latent_dim).to(device)


for rest, task in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs} [train]"):
            # combine rest + task to train a shared AE
    X = torch.cat([rest, task], dim=0).to(device)  # (2,1,H,W,D)

    z, feat_shape = encoder(X)
    X_hat = decod

# AE Training

In [14]:

# ===========================================
# 1. Dataset with NaN cleaning + AAL mask
# ===========================================

class HCPRestTaskDataset(Dataset):
    def __init__(
        self,
        rest_dir,
        task_dir,
        subject_ids,
        mask_np=None,
        normalize="zscore",
        global_mean=None, global_std=None
    ):
        self.rest_dir = rest_dir
        self.task_dir = task_dir
        self.subjects = list(subject_ids)
        self.normalize = normalize
        self.global_mean = global_mean
        self.global_std = global_std


        # broadcast mask to (1,1,H,W,D) later
        self.mask_np = mask_np.astype(np.float32) if mask_np is not None else None

        print(f"[Dataset] Loaded {len(self.subjects)} subjects")

    def __len__(self):
        return len(self.subjects)

    def _clean(self, img):
        """Remove NaN, +Inf, -Inf."""
        return np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)

#     def _normalize(self, img):
#         img = self._clean(img)
#         m, s = img.mean(), img.std()
#         if s < 1e-6:
#             return np.zeros_like(img)
#         return (img - m) / s
    
    def _normalize(self, img):
        img = self._clean(img)
        if self.global_mean is None:
            m, s = img.mean(), img.std()
        else:
            m, s = self.global_mean, self.global_std
        if s < 1e-6:
            return np.zeros_like(img)
        return (img - m) / s

    def __getitem__(self, idx):
        sid = self.subjects[idx]

        rest = nib.load(os.path.join(self.rest_dir, f"{sid}_fALFF.nii")).get_fdata()
        task = nib.load(os.path.join(self.task_dir, f"{sid}_2bk-0bk.nii")).get_fdata()

        rest = self._clean(rest)
        task = self._clean(task)

        if self.mask_np is not None:
            rest = rest * self.mask_np
            task = task * self.mask_np

        rest = self._normalize(rest)
        task = self._normalize(task)

        rest = torch.tensor(rest, dtype=torch.float32)[None, ...]  # (1,H,W,D)
        task = torch.tensor(task, dtype=torch.float32)[None, ...]

        return rest, task


# ===========================================
# 2. Stable Encoder & Decoder (no flatten!)
# ===========================================

# class Encoder3D(nn.Module):
#     def __init__(self, latent_dim=128):
#         super().__init__()
#         self.conv = nn.Sequential(
#             nn.Conv3d(1, 32, 3, stride=2, padding=1), nn.LeakyReLU(0.01, inplace=True),
#             nn.Conv3d(32, 64, 3, stride=2, padding=1), nn.LeakyReLU(0.01, inplace=True),
#             nn.Conv3d(64, 128, 3, stride=2, padding=1), nn.LeakyReLU(0.01, inplace=True),
#             nn.Conv3d(128, 256, 3, stride=2, padding=1), nn.LeakyReLU(0.01, inplace=True),
#         )
#         self.to_latent = nn.Conv3d(256, latent_dim, kernel_size=1)

#     def forward(self, x):
#         h = self.conv(x)
#         h_lat = self.to_latent(h)
#         #z = torch.mean(h_lat, dim=[2,3,4])
#         h = self.conv(x)                 # (B,C,H',W',D')
#         h_flat = h.view(B, -1)
#         z = self.fc_mu(h_flat)           # (B, latent_dim)

#         return z, h_lat.shape

# class Decoder3D(nn.Module):
#     def __init__(self, latent_dim=128):
#         super().__init__()
#         self.expand = nn.ConvTranspose3d(latent_dim, 256, 4, stride=1)
#         self.deconv = nn.Sequential(
#             nn.ConvTranspose3d(256, 128, 4, stride=2, padding=1), nn.LeakyReLU(0.01, inplace=True),
#             nn.ConvTranspose3d(128, 64, 4, stride=2, padding=1),  nn.LeakyReLU(0.01, inplace=True),
#             nn.ConvTranspose3d(64, 32, 4, stride=2, padding=1),   nn.LeakyReLU(0.01, inplace=True),
#             nn.ConvTranspose3d(32, 1, 4, stride=2, padding=1),
#         )
#     def forward(self, z, orig_shape):
#         B, _, H0, W0, D0 = orig_shape
#         h = z[:, :, None, None, None]
#         h = self.expand(h)
#         x_hat = self.deconv(h)
#         x_hat = F.interpolate(x_hat, size=(H0, W0, D0),
#                               mode="trilinear", align_corners=False)
#         return x_hat


# # ===========================================
# # 3. Load subject ID list
# # ===========================================

# def load_ids(path):
#     with open(path, "r") as f:
#         return [line.strip() for line in f.readlines()]


# ===========================================
# 4. Masked MSE Loss
# ===========================================

# def masked_mse(X_hat, X, mask_t):
#     """
#     Compute MSE only inside mask.
#     mask_t: (1,1,H,W,D) tensor
#     """
#     err = (X_hat - X) ** 2
#     masked_err = err * mask_t
#     return masked_err.sum() / (mask_t.sum() + 1e-6)

def masked_mse(X_hat, X, mask_t):
    """
    Batched masked MSE:
        Returns mean over batch of (MSE on masked region).
    X_hat: (B,1,H,W,D)
    X:     (B,1,H,W,D)
    mask_t: (1,1,H,W,D)
    """
    # Broadcast mask to batch
    mask = mask_t   # shape (1,1,H,W,D), auto-broadcasts

    # compute squared error inside mask
    err = (X_hat - X) ** 2              # (B,1,H,W,D)
    masked_err = err * mask             # (B,1,H,W,D)

    # sum error over spatial dims for each batch element
    spatial_error = masked_err.view(X.shape[0], -1).sum(dim=1)  # (B,)

    # number of valid voxels (same for each batch element)
    denom = mask.sum() + 1e-6

    # MSE per subject
    mse_per_subject = spatial_error / denom   # (B,)

    # average over batch
    return mse_per_subject.mean()



def compute_global_stats(rest_dir, task_dir, train_ids, mask_np):
    vals = []
    for sid in train_ids:
        rest = nib.load(os.path.join(rest_dir, f"{sid}_fALFF.nii")).get_fdata()
        task = nib.load(os.path.join(task_dir, f"{sid}_2bk-0bk.nii")).get_fdata()
        rest = np.nan_to_num(rest)
        task = np.nan_to_num(task)

        if mask_np is not None:
            rest = rest * mask_np
            task = task * mask_np

        vals.append(rest.flatten())
        vals.append(task.flatten())

    vals = np.concatenate(vals)
    mu = vals.mean()
    std = vals.std()
    np.save("global_mean.npy", mu)
    np.save("global_std.npy", std)
    return mu, std


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ResidualBlock3D(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv3d(channels, channels, kernel_size=3, padding=1)
        self.norm1 = nn.InstanceNorm3d(channels)
        self.conv2 = nn.Conv3d(channels, channels, kernel_size=3, padding=1)
        self.norm2 = nn.InstanceNorm3d(channels)
        self.act = nn.LeakyReLU(0.01, inplace=True)

    def forward(self, x):
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.act(out)

        out = self.conv2(out)
        out = self.norm2(out)

        out = out + x
        out = self.act(out)
        return out


class DownBlock3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.down = nn.Conv3d(in_ch, out_ch, kernel_size=3, stride=2, padding=1)
        self.norm = nn.InstanceNorm3d(out_ch)
        self.act = nn.LeakyReLU(0.01, inplace=True)
        self.res = ResidualBlock3D(out_ch)

    def forward(self, x):
        x = self.down(x)
        x = self.norm(x)
        x = self.act(x)
        x = self.res(x)
        return x


class UpBlock3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose3d(in_ch, out_ch, kernel_size=4, stride=2, padding=1)
        self.norm = nn.InstanceNorm3d(out_ch)
        self.act = nn.LeakyReLU(0.01, inplace=True)
        self.res = ResidualBlock3D(out_ch)

    def forward(self, x):
        x = self.up(x)
        x = self.norm(x)
        x = self.act(x)
        x = self.res(x)
        return x


In [29]:
class Encoder3DStrong(nn.Module):
    def __init__(self, latent_dim=128, base_channels=16):
        super().__init__()
        self.latent_dim = latent_dim
        self.base = base_channels

        # Initial conv + residual
        self.stem = nn.Sequential(
            nn.Conv3d(1, base_channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm3d(base_channels),
            nn.LeakyReLU(0.01, inplace=True),
            ResidualBlock3D(base_channels),
        )

        # Downsampling path
        self.down1 = DownBlock3D(base_channels, base_channels * 2)   # 16 -> 32
        self.down2 = DownBlock3D(base_channels * 2, base_channels * 4)  # 32 -> 64
        self.down3 = DownBlock3D(base_channels * 4, base_channels * 8)  # 64 -> 128

        # Latent projection
        self.fc_mu = None  # initialized lazily after seeing first input

    def _build_fc(self, feature_shape):
        """
        Called once after we know the conv output spatial size.
        feature_shape = (C, H, W, D)
        """
        C, H, W, D = feature_shape
        self.flat_dim = C * H * W * D
        self.fc_mu = nn.Linear(self.flat_dim, self.latent_dim)
        # optional: weight init
        nn.init.xavier_uniform_(self.fc_mu.weight)
        self.fc_mu.to(next(self.parameters()).device)

    def forward(self, x):
        """
        x: (B,1,H,W,D)
        returns:
          z: (B, latent_dim)
          feature_shape: (C, H', W', D') needed by decoder
        """
        h = self.stem(x)          # (B, C, H, W, D)
        h = self.down1(h)         # (B, 2C, H/2, W/2, D/2)
        h = self.down2(h)         # (B, 4C, ...)
        h = self.down3(h)         # (B, 8C, ...)

        B, C, H, W, D = h.shape
        feature_shape = (C, H, W, D)

        # lazy init fc layer (so it adapts to actual spatial size)
        if self.fc_mu is None:
            self._build_fc(feature_shape)

        h_flat = h.view(B, -1)
        z = self.fc_mu(h_flat)
        return z, feature_shape


In [30]:
class Decoder3DStrong(nn.Module):
    def __init__(self, latent_dim, feature_shape, out_spatial_shape):
        """
        feature_shape: (C, Hf, Wf, Df) from encoder
        out_spatial_shape: (H0, W0, D0) = original fMRI volume size
        """
        super().__init__()

        C, Hf, Wf, Df = feature_shape
        self.feature_shape = (C, Hf, Wf, Df)
        self.flat_dim = C * Hf * Wf * Df

        H0, W0, D0 = out_spatial_shape
        self.out_spatial_shape = (H0, W0, D0)

        # latent → feature
        self.fc = nn.Linear(latent_dim, self.flat_dim)

        self.up = nn.Sequential(
            nn.ConvTranspose3d(C, 128, 3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(0.01, inplace=True),

            nn.ConvTranspose3d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(0.01, inplace=True),

            nn.ConvTranspose3d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(0.01, inplace=True),

            nn.Conv3d(32, 1, 3, padding=1),
        )

    def forward(self, z, target_shape=None):
        """
        z: (B, latent_dim)
        target_shape: optional (B,1,H,W,D). If None, we use self.out_spatial_shape.
        """
        B = z.size(0)

        h = self.fc(z)
        h = h.view(B, *self.feature_shape)   # (B, C, Hf, Wf, Df)
        x_hat = self.up(h)                   # (B, 1, H*, W*, D*)

        # --- always interpolate to desired output size ---
        if target_shape is not None:
            _, _, H, W, D = target_shape
        else:
            H, W, D = self.out_spatial_shape

        x_hat = F.interpolate(
            x_hat,
            size=(H, W, D),
            mode="trilinear",
            align_corners=False,
        )

        return x_hat


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# ----------------------------------------
# Basic U-Net building blocks (3D)
# ----------------------------------------

class DoubleConv3D(nn.Module):
    """
    (Conv3d -> InstanceNorm3d -> LeakyReLU) * 2
    """
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.InstanceNorm3d(out_ch),
            nn.LeakyReLU(0.01, inplace=True),

            nn.Conv3d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.InstanceNorm3d(out_ch),
            nn.LeakyReLU(0.01, inplace=True),
        )

    def forward(self, x):
        return self.net(x)


class Down3D(nn.Module):
    """
    Downsampling: MaxPool3d(2) + DoubleConv3D
    """
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.conv = DoubleConv3D(in_ch, out_ch)

    def forward(self, x):
        x = self.pool(x)
        x = self.conv(x)
        return x


def center_crop_3d(x, target_dhw):
    """
    Center-crop a 5D tensor (B,C,D,H,W) to target spatial size (D_t,H_t,W_t).
    This handles small off-by-one mismatches from upsampling / pooling.
    """
    _, _, D, H, W = x.shape
    D_t, H_t, W_t = target_dhw

    d_start = max((D - D_t) // 2, 0)
    h_start = max((H - H_t) // 2, 0)
    w_start = max((W - W_t) // 2, 0)

    d_end = d_start + D_t
    h_end = h_start + H_t
    w_end = w_start + W_t

    return x[:, :, d_start:d_end, h_start:h_end, w_start:w_end]


class Up3D(nn.Module):
    """
    Upsampling block: ConvTranspose3d + concat skip + DoubleConv3D
    """
    def __init__(self, in_ch, out_ch):
        """
        in_ch: channels of feature BEFORE concat (from decoder)
        out_ch: output channels AFTER DoubleConv
        """
        super().__init__()
        # Reduce spatial size mismatch via transposed conv
        self.up = nn.ConvTranspose3d(
            in_ch, in_ch // 2,
            kernel_size=2, stride=2
        )
        self.conv = DoubleConv3D(in_ch, out_ch)  # (decoder_ch + skip_ch)

    def forward(self, x, skip):
        # x: decoder feature, skip: encoder feature
        x = self.up(x)        # (B, in_ch//2, D*,H*,W*)

        # crop skip to match x spatial size
        _, _, D, H, W = x.shape
        skip_c = center_crop_3d(skip, (D, H, W))

        # concat along channels
        x = torch.cat([x, skip_c], dim=1)  # (B, in_ch, D,H,W)
        x = self.conv(x)
        return x


class OutConv3D(nn.Module):
    def __init__(self, in_ch, out_ch=1):
        super().__init__()
        self.conv = nn.Conv3d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


In [ ]:
class UNet3DAE(nn.Module):
    """
    3D U-Net style autoencoder for fMRI volumes.
    Returns:
      x_hat: reconstruction (B,1,D,H,W)
      z: latent vector (B, latent_dim) from bottleneck features
    """
    def __init__(self, in_channels=1, base_ch=16, latent_dim=128):
        super().__init__()
        self.in_channels = in_channels
        self.base_ch = base_ch
        self.latent_dim = latent_dim

        # ------------- Encoder (U-Net down path) -------------
        self.inc   = DoubleConv3D(in_channels, base_ch)         # level 0
        self.down1 = Down3D(base_ch,       base_ch * 2)         # level 1
        self.down2 = Down3D(base_ch * 2,   base_ch * 4)         # level 2
        self.down3 = Down3D(base_ch * 4,   base_ch * 8)         # level 3
        # one more down for bottleneck
        self.down4 = Down3D(base_ch * 8,   base_ch * 16)        # bottleneck

        # ------------- Latent head (for SB, etc.) -------------
        self.global_pool = nn.AdaptiveAvgPool3d(1)  # (B,C,1,1,1)
        self.fc_mu = nn.Linear(base_ch * 16, latent_dim)

        # ------------- Decoder (U-Net up path) -------------
        self.up1 = Up3D(base_ch * 16, base_ch * 8)  # bottleneck + skip3
        self.up2 = Up3D(base_ch * 8,  base_ch * 4)  # dec + skip2
        self.up3 = Up3D(base_ch * 4,  base_ch * 2)  # dec + skip1
        self.up4 = Up3D(base_ch * 2,  base_ch)      # dec + skip0

        self.outc = OutConv3D(base_ch, 1)

    def encode(self, x):
        """
        Encoder only: returns latent z and bottleneck feature for debugging.
        """
        x0 = self.inc(x)       # (B, base,   D0,H0,W0)
        x1 = self.down1(x0)    # (B, 2base,  D1,H1,W1)
        x2 = self.down2(x1)    # (B, 4base,  D2,H2,W2)
        x3 = self.down3(x2)    # (B, 8base,  D3,H3,W3)
        x4 = self.down4(x3)    # (B,16base,  D4,H4,W4) bottleneck

        # latent vector from bottleneck
        pooled = self.global_pool(x4).view(x4.size(0), -1)  # (B, 16*base)
        z = self.fc_mu(pooled)                              # (B, latent_dim)
        return z, (x0, x1, x2, x3, x4)

    def decode(self, x0, x1, x2, x3, x4, out_shape):
        """
        Decoder given encoder feature maps.
        out_shape: (B,1,D0,H0,W0) of original input, used to final-resize.
        """
        x = self.up1(x4, x3)      # (B,8base, D3',H3',W3')
        x = self.up2(x,  x2)      # (B,4base, D2',H2',W2')
        x = self.up3(x,  x1)      # (B,2base, D1',H1',W1')
        x = self.up4(x,  x0)      # (B,base,  D0',H0',W0')

        x = self.outc(x)          # (B,1,D*,H*,W*)

        # Final interpolation to EXACT input size (handles odd dims like 91x109x91)
        _, _, D0, H0, W0 = out_shape
        if (x.shape[2] != D0) or (x.shape[3] != H0) or (x.shape[4] != W0):
            x = F.interpolate(
                x,
                size=(D0, H0, W0),
                mode="trilinear",
                align_corners=False,
            )
        return x

    def forward(self, x):
        """
        Full AE: input -> reconstruction + latent.
        x: (B,1,D,H,W)
        returns:
          x_hat: (B,1,D,H,W)
          z:     (B, latent_dim)
        """
        B, C, D0, H0, W0 = x.shape

        z, (x0, x1, x2, x3, x4) = self.encode(x)
        x_hat = self.decode(x0, x1, x2, x3, x4, x.shape)
        return x_hat, z


In [17]:
# ===========================================
# 5. Main AE Training Function
# ===========================================

def main():
    # -------- Paths --------
    rest_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/"
    task_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/"
    mask_path = "/nfs/turbo/sph-jianghui1/yutingd/HCP/AAL_MNI_2mm.nii"

    train_ids = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/training_ids.txt")
    val_ids   = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/validation_ids.txt")

    # -------- Load Mask --------
    mask_np = nib.load(mask_path).get_fdata()
    mask_np = (mask_np > 0).astype(np.float32)

    # -------- Device --------
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # -------- Dataset & Dataloaders --------
    mu, std = compute_global_stats(rest_dir, task_dir, train_ids, mask_np)
    train_ds = HCPRestTaskDataset(rest_dir, task_dir, train_ids, mask_np=mask_np,
                              normalize="zscore", global_mean=mu, global_std=std)

#     train_ds = HCPRestTaskDataset(
#         rest_dir, task_dir, train_ids, mask_np=mask_np
#     )
    val_ds = HCPRestTaskDataset(
        rest_dir, task_dir, val_ids, mask_np=mask_np,normalize="zscore", global_mean=mu, global_std=std
    )

    train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False)

    # -------- Mask tensor for loss --------
    mask_t = torch.tensor(mask_np, dtype=torch.float32, device=device)[None, None]

    # -------- AE Models --------
    latent_dim = 128
    encoder = Encoder3DStrong(latent_dim).to(device)

    # run 1 sample through encoder to get feature_shape
    rest, task = next(iter(train_loader))
    #X = torch.cat([rest, task], dim=0).to(device)
    X = task.to(device)


    with torch.no_grad():
        z, feature_shape = encoder(X)
    _, _, H0, W0, D0 = X.shape

    decoder = Decoder3DStrong(
        latent_dim,
        feature_shape=feature_shape,
        out_spatial_shape=(H0, W0, D0),
    ).to(device)
    #decoder = Decoder3DStrong(latent_dim, feature_shape).to(device)
    #encoder = Encoder3D(latent_dim).to(device)
    #decoder = Decoder3D(latent_dim).to(device)
#     encoder = Encoder3DStrong(latent_dim=128, base_channels=16).to(device)
#     decoder = Decoder3DStrong(latent_dim=128, base_channels=16).to(device)
    params = list(encoder.parameters()) + list(decoder.parameters())
    optimizer = torch.optim.AdamW(params, lr=5e-4, weight_decay=1e-5)


    #params = list(encoder.parameters()) + list(decoder.parameters())
    #optimizer = torch.optim.Adam(params, lr=1e-3)
    scaler = GradScaler()

#     latent_dim = 128
#     ae = UNet3DAE(in_channels=1, base_ch=16, latent_dim=latent_dim).to(device)

#     params = list(ae.parameters())
#     optimizer = torch.optim.AdamW(params, lr=5e-4, weight_decay=1e-5)
#     scaler = GradScaler()


    best_val = float("inf")
    best_train = float("inf")
    epochs = 50

    # -------- Training Loop --------
    for epoch in range(1, epochs+1):
        encoder.train()
        decoder.train()
        train_loss = 0

        for rest, task in tqdm(train_loader, desc=f"Epoch {epoch} [train]"):
            #X = torch.cat([rest, task], dim=0).to(device)  # (2,1,H,W,D)
            X = task.to(device)

            optimizer.zero_grad()

            with autocast():
                z, feat_shape = encoder(X)
                #X_hat = decoder(z, X.shape)
                X_hat = decoder(z) 

                loss = masked_mse(X_hat, X, mask_t)
                  

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        # -------- Save best --------
        if train_loss < best_train:
            best_train = train_loss
            torch.save(encoder.state_dict(), "encoder_ae_train.pth")
            torch.save(decoder.state_dict(), "decoder_ae_train.pth")
            print("  → Saved new best AE train model")


        # -------- Validation --------
        encoder.eval()
        decoder.eval()
        val_loss = 0

        with torch.no_grad():
            for rest, task in tqdm(val_loader, desc=f"Epoch {epoch} [val]"):
                #X = torch.cat([rest, task], dim=0).to(device)
                X = task.to(device)

                with autocast():
                    z, feat_shape = encoder(X)
                    X_hat = decoder(z)
                    #X_hat, z = ae(X)
                    loss = masked_mse(X_hat, X, mask_t)

                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f"Epoch {epoch} | train={train_loss:.4f} | val={val_loss:.4f}")
        
        # -------- Save best --------
        if val_loss < best_val:
            best_val = val_loss
            torch.save(encoder.state_dict(), "encoder_ae_best.pth")
            torch.save(decoder.state_dict(), "decoder_ae_best.pth")
            print("  → Saved new best AE model")

    print("Training complete. Best val:", best_val)


if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [136]:
rest, task = next(iter(test_loader))
task = task.to(device)
#rest = rest.to(device)
mask_np = nib.load(mask_path).get_fdata()
mask_np = (mask_np > 0).astype(np.float32)
mask_t  = torch.tensor(mask_np, dtype=torch.float32)[None, None]
mask_t = mask_t.to(device)

print(mask_t.shape)
print(task.shape)

if mask_t.shape[0] != task.shape[0]:
    mask_batched = mask_t.repeat(task.shape[0], 1, 1, 1, 1)
else:
    mask_batched = mask_t
with torch.no_grad():
    z, _ = encoder(task)
    task_recon = decoder(z, task.shape)
    
    #zr, _ = encoder(rest)
    #rest_recon = decoder(zr, rest.shape)
    
mse = masked_mse(task_recon, task, mask_batched).item()
corr = masked_corr(task_recon, task, mask_batched).item()
ssim = masked_ssim_3d(task_recon, task, mask_batched).item()
print("MSE:", mse)
print("corr:", corr)
print("ssim:", ssim)


print("Task true  mean/std:", task.mean().item(), task.std().item())
print("Task recon mean/std:", task_recon.mean().item(), task_recon.std().item())

    
#print("Rest true  mean/std:", rest.mean().item(), rest.std().item())
#print("Rest recon mean/std:", rest_recon.mean().item(), rest_recon.std().item())

print("Ratio (recon/true):", 
      task_recon.mean().item()/task.mean().item() if task.mean().item()!=0 else None)

torch.Size([1, 1, 91, 109, 91])
torch.Size([1, 1, 91, 109, 91])
MSE: 3.5825729370117188
corr: 0.5495498180389404
ssim: 0.11813586950302124
Task true  mean/std: 4.3276364714017745e-09 1.0000005960464478
Task recon mean/std: -0.35254940390586853 0.8404914736747742
Ratio (recon/true): -81464653.10467112


In [137]:
rest_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/"
task_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/"
mask_path = "/nfs/turbo/sph-jianghui1/yutingd/HCP/AAL_MNI_2mm.nii"

train_ids = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/training_ids.txt")
val_ids   = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/validation_ids.txt")

test_ids  = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/test_ids.txt")

device = "cuda" if torch.cuda.is_available() else "cpu"

# -------- Load mask --------
mask_np = nib.load(mask_path).get_fdata()
mask_np = (mask_np > 0).astype(np.float32)
mask_t = torch.tensor(mask_np, dtype=torch.float32, device=device)[None, None]

# -------- Datasets & loaders --------
train_ds = HCPRestTaskDataset(rest_dir, task_dir, train_ids, mask_np=mask_np)
val_ds   = HCPRestTaskDataset(rest_dir, task_dir, val_ids, mask_np=mask_np)
test_ds  = HCPRestTaskDataset(rest_dir, task_dir, test_ids, mask_np=mask_np)

B = 16
train_loader = DataLoader(train_ds, batch_size=B, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=B, shuffle=False, drop_last=True)
test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False)

# -------- Load trained AE --------
encoder = Encoder3DStrong(latent_dim).to(device)

# run 1 sample through encoder to get feature_shape
rest, task = next(iter(train_loader))
#X = torch.cat([rest, task], dim=0).to(device)
X = task.to(device)


with torch.no_grad():
    z, feature_shape = encoder(X)
_, _, H0, W0, D0 = X.shape

decoder = Decoder3DStrong(
    latent_dim,
    feature_shape=feature_shape,
    out_spatial_shape=(H0, W0, D0),
).to(device)
encoder.load_state_dict(torch.load("encoder_ae_best.pth", map_location=device))
decoder.load_state_dict(torch.load("decoder_ae_best.pth", map_location=device))

encoder.eval()
decoder.eval()
for p in encoder.parameters():
    p.requires_grad = False
for p in decoder.parameters():
    p.requires_grad = False

# epoch = 1
# for rest, task in tqdm(train_loader, desc=f"SB Epoch {epoch} [train]"):
#     rest = rest.to(device)   # (1,1,H,W,D)
#     task = task.to(device)

#     with torch.no_grad():
#         z, _ = encoder(rest)  # (1,d)
#         rest_recon = decoder(z, task.shape)
        
        
#         z_r,_ = encoder(rest)
#         z_t,_ = encoder(task)

#         print("Rest latent norm:", z_r.norm(dim=1).mean().item())
#         print("Task latent norm:", z_t.norm(dim=1).mean().item())



[Dataset] Loaded 499 subjects
[Dataset] Loaded 142 subjects
[Dataset] Loaded 73 subjects


/tmp/ipykernel_1754538/3782407550.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("encoder_ae_best.pth", map_location=device))
/tmp/i

In [28]:
print("Task true  mean/std:", rest.mean().item(), rest.std().item())
print("Rest recon mean/std:", rest_recon.mean().item(), rest_recon.std().item())
print("Ratio (recon/true):", 
      rest_recon.mean().item()/task.mean().item() if task.mean().item()!=0 else None)

Task true  mean/std: -1.0142897632903214e-09 1.0
Rest recon mean/std: 0.5522738099098206 1.1539183855056763
Ratio (recon/true): -82917735.36789353


In [32]:
encoder.eval()
all_z = []

with torch.no_grad():
    for rest, task in train_loader:
        rest = rest.to(device)
        task = task.to(device)

        z_r, _ = encoder(rest)
        z_t, _ = encoder(task)

        all_z.append(z_r.cpu())
        all_z.append(z_t.cpu())

all_z = torch.cat(all_z, dim=0)        # (N_total, latent_dim)
z_mean = all_z.mean(dim=0)            # (latent_dim,)
z_std  = all_z.std(dim=0) + 1e-6      # avoid zeros

torch.save({"mean": z_mean, "std": z_std}, "latent_stats.pt")


In [33]:
print(z_mean)
print(z_std)

tensor([ 1.5712,  3.1305, -4.8395,  4.4951,  0.9114, -5.6921,  0.2736,  2.9826,
        -0.0211,  0.3143, -3.0756, -4.0493,  0.5063, -3.2029,  5.6924, -2.8474,
         2.9180,  1.9802,  5.1454, -3.6219,  3.9465, -2.0529, -0.5626, -3.6194,
        -1.1389, -4.2627, -2.6841, -5.2748, -6.5108, -0.7909,  0.0553,  3.6960,
        -3.7562,  4.7947,  3.6051, -2.8303,  6.8552, -4.8011,  1.0891, -3.3245,
         2.5340,  5.5578,  2.9018,  3.5488, -6.2206, -3.6326, -5.7298, -3.6517,
         1.8035,  7.1596,  0.4957, -2.8218,  5.7257,  4.6408, -3.5853, -2.0040,
         2.8357,  3.3378, -3.9196,  0.3642, -4.4083, -5.4960,  0.2924, -0.8236,
         0.1083, -0.1733, -3.1912,  2.5541, -0.2050, -5.7374,  1.4797,  2.9771,
        -5.0296,  0.9904,  3.8877, -1.3343,  4.2282,  0.5186,  6.0291, -5.7672,
         4.1937,  3.2922,  0.6151,  1.3207, -3.2521,  3.7449,  1.4001, -1.5807,
         2.6302,  6.8396,  3.1028, -3.4176, -2.5884,  3.3322, -2.3514,  3.7217,
         2.1221,  3.9591,  3.3379,  1.27

# SB Training

In [18]:
def masked_corr(X_hat, X, mask_t, eps=1e-6):
    # X_hat, X: (1,1,H,W,D), mask_t: (1,1,H,W,D)
    mask_flat = mask_t.view(-1) > 0
    x = X_hat.view(-1)[mask_flat]
    y = X.view(-1)[mask_flat]

    x = x - x.mean()
    y = y - y.mean()
    num = (x * y).sum()
    den = torch.sqrt((x**2).sum() * (y**2).sum() + eps)
    return num / den


# ===========================================
# 4. Drift network & flow-matching loss
# ===========================================

class DriftNet(nn.Module):
    """
    b_phi(z_t, t, z0): latent drift field for SB / flow-matching.
    """
    def __init__(self, latent_dim=128, hidden=256):
        super().__init__()
        self.fc_t = nn.Linear(1, 16)

        self.net = nn.Sequential(
            nn.Linear(latent_dim * 2 + 16, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, latent_dim),
        )

    def forward(self, zt, t, z0):
        """
        zt, z0: (B,latent_dim)
        t: (B,) in [0,1]
        """
        t_emb = self.fc_t(t.unsqueeze(1))  # (B,16)
        h = torch.cat([zt, z0, t_emb], dim=1)
        return self.net(h)


def flow_matching_loss(z0_n, z1_n, drift_net, sigma=0.1):
    B, d = z0_n.shape
    device = z0_n.device

    t = torch.rand(B, device=device)  # (B,)
    eps = torch.randn_like(z0_n)

    # sample along noisy interpolant
    zt_n = (1 - t[:, None]) * z0_n + t[:, None] * z1_n + sigma * eps

    v_true = z1_n - z0_n
    v_pred = drift_net(zt_n, t, z0_n)

    return F.mse_loss(v_pred, v_true)

# ===========================================
# 5. ODE integration (Euler) in latent space
# ===========================================

def euler_integrate(z0_n, drift_net, steps=20):
    z = z0_n.clone()
    dt = 1.0 / steps
    for i in range(steps):
        t = torch.full((z.size(0),), i * dt, device=z.device)
        dz = drift_net(z, t, z0_n)
        z = z + dt * dz
    return z

In [139]:
# ===========================================
# 6. Main: SB training + evaluation
# ===========================================
def main():
    # -------- Paths --------
    rest_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/"
    task_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/"
    mask_path = "/nfs/turbo/sph-jianghui1/yutingd/HCP/AAL_MNI_2mm.nii"

    train_ids = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/training_ids.txt")
    val_ids   = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/validation_ids.txt")

    test_ids  = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/test_ids.txt")

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # -------- Load mask --------
    mask_np = nib.load(mask_path).get_fdata()
    mask_np = (mask_np > 0).astype(np.float32)
    mask_t = torch.tensor(mask_np, dtype=torch.float32, device=device)[None, None]

    # -------- Datasets & loaders --------
    train_ds = HCPRestTaskDataset(rest_dir, task_dir, train_ids, mask_np=mask_np)
    val_ds   = HCPRestTaskDataset(rest_dir, task_dir, val_ids, mask_np=mask_np)
    test_ds  = HCPRestTaskDataset(rest_dir, task_dir, test_ids, mask_np=mask_np)
    
    B = 16
    train_loader = DataLoader(train_ds, batch_size=B, shuffle=True, drop_last=True)
    val_loader   = DataLoader(val_ds, batch_size=B, shuffle=False, drop_last=True)
    test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False)

    # -------- Load trained AE --------
    latent_dim = 128
    # -------- Load trained AE --------
    encoder = Encoder3DStrong(latent_dim).to(device)

    # run 1 sample through encoder to get feature_shape
    rest, task = next(iter(train_loader))
    #X = torch.cat([rest, task], dim=0).to(device)
    X = task.to(device)


    with torch.no_grad():
        z, feature_shape = encoder(X)
    _, _, H0, W0, D0 = X.shape

    decoder = Decoder3DStrong(
        latent_dim,
        feature_shape=feature_shape,
        out_spatial_shape=(H0, W0, D0),
    ).to(device)
    encoder.load_state_dict(torch.load("encoder_ae_best.pth", map_location=device))
    decoder.load_state_dict(torch.load("decoder_ae_best.pth", map_location=device))

    encoder.eval()
    decoder.eval()
    for p in encoder.parameters():
        p.requires_grad = False
    for p in decoder.parameters():
        p.requires_grad = False

    print("[SB] Loaded pretrained AE.")

    # -------- Drift network for SB --------
    drift_net = DriftNet(latent_dim=latent_dim, hidden=256).to(device)
    optimizer = torch.optim.Adam(drift_net.parameters(), lr=1e-4)
    #scaler = GradScaler()
    scaler = torch.cuda.amp.GradScaler()

    epochs_sb = 30
    sigma = 0.3
    best_val = float("inf")
    
    stats = torch.load("latent_stats.pt")
    z_mean = stats["mean"].to(device)
    z_std  = stats["std"].to(device)


    # -------- SB training (latent flow-matching) --------
    for epoch in range(1, epochs_sb + 1):
        drift_net.train()
        train_loss = 0.0

        for rest, task in tqdm(train_loader, desc=f"SB Epoch {epoch} [train]"):
            rest = rest.to(device)   # (1,1,H,W,D)
            task = task.to(device)

            with torch.no_grad():
                # ---- raw latents ----
                z_r, _ = encoder(rest)
                z_t, _ = encoder(task)

                # ---- normalize latents ----
                z_r_n = (z_r - z_mean) / z_std
                z_t_n = (z_t - z_mean) / z_std


            optimizer.zero_grad()
            with autocast():
                #loss = flow_matching_loss(z_r, z_t, drift_net, sigma=sigma)
                # FM core
                fm_loss = flow_matching_loss(z_r_n, z_t_n,  drift_net, sigma=sigma)

                # predicted midpoint latent in normalized space
                z_mid_n = (z_r_n + z_t_n) / 2

                # unnormalize
                z_mid = z_mid_n * z_std + z_mean
                z_t_un = z_t_n * z_std + z_mean

                # decode
                x_mid = decoder(z_mid, rest.shape)
                X_pred = decoder(z_t_un, task.shape)

                # ===========================
                # 3) IMAGE-SPACE LOSSES
                # ===========================
                task_loss = masked_mse(X_pred, task, mask_t)
                recon_loss = masked_mse(x_mid, task, mask_t)

                scale_loss = torch.mean(x_mid**2)

                loss = fm_loss + task_loss + 0.1 * recon_loss + 1e-4 * scale_loss


            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(drift_net.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # ---- validation: use flow-matching loss on val set ----
        drift_net.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rest, task in tqdm(val_loader, desc=f"SB Epoch {epoch} [val]"):
                rest = rest.to(device)
                task = task.to(device)

                z_r, _ = encoder(rest)
                z_t, _ = encoder(task)

                z_r_n = (z_r - z_mean) / z_std
                z_t_n = (z_t - z_mean) / z_std

                fm_loss = flow_matching_loss(z_r_n, z_t_n, drift_net, sigma=sigma)

                z_mid_n = (z_r_n + z_t_n) / 2
                z_mid = z_mid_n * z_std + z_mean
                x_mid = decoder(z_mid, rest.shape)

                z_t_un = z_t_n * z_std + z_mean
                X_pred = decoder(z_t_un, task.shape)

                recon_loss = masked_mse(x_mid, task, mask_t)
                task_loss  = masked_mse(X_pred, task, mask_t)
                scale_loss = torch.mean(x_mid**2)

                loss = fm_loss + task_loss + 0.1 * recon_loss + 1e-4 * scale_loss

                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f"[SB] Epoch {epoch}: train={train_loss:.4f}, val={val_loss:.4f}")

        if val_loss < best_val:
            best_val = val_loss
            torch.save(drift_net.state_dict(), "driftnet_sb_best.pth")
            print("  → Saved new best SB drift model")

    print("[SB] Training complete. Best val loss:", best_val)

    

if __name__ == "__main__":
    main()

[Dataset] Loaded 499 subjects
[Dataset] Loaded 142 subjects
[Dataset] Loaded 73 subjects


/tmp/ipykernel_1754538/2610987264.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("encoder_ae_best.pth", map_location=device))
/tmp/i

[SB] Loaded pretrained AE.


SB Epoch 1 [train]:   0%|          | 0/31 [00:00<?, ?it/s]/tmp/ipykernel_1754538/2610987264.py:99: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
SB Epoch 1 [val]: 100%|██████████| 8/8 [00:11<00:00,  1.47s/it]


[SB] Epoch 1: train=64857.4858, val=59572.6943
  → Saved new best SB drift model


...

[SB] Epoch 28: train=11399.9847, val=9406.2609


SB Epoch 29 [val]: 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]


[SB] Epoch 29: train=11330.8453, val=8254.4481
  → Saved new best SB drift model


SB Epoch 30 [val]: 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]

[SB] Epoch 30: train=10837.0811, val=9432.7264
[SB] Training complete. Best val loss: 8254.448059082031


In [140]:
z0, _ = encoder(rest)
z0 = z0.to(device)
z_mean = z_mean.to(device)
z_std = z_std.to(device)
z0_n = (z0 - z_mean) / z_std
z0_u = z0_n * z_std + z_mean
X_test = decoder(z0_u, rest.shape)
print(X_test)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [141]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cuda


In [19]:
def masked_corr(X_hat, X, mask_t, eps=1e-6):
    """
    Compute voxelwise correlation only in masked region.
    Works for non-contiguous tensors.
    """
    mask_flat = mask_t.reshape(-1) > 0
    x = X_hat.reshape(-1)[mask_flat]
    y = X.reshape(-1)[mask_flat]

    if x.numel() < 2:
        return torch.tensor(0.0, device=X.device)

    x = x - x.mean()
    y = y - y.mean()

    num = (x * y).sum()
    den = torch.sqrt((x**2).sum() * (y**2).sum() + eps)
    return num / den


In [20]:
import torch
import torch.nn.functional as F

def gaussian_kernel_3d(channels, kernel_size=11, sigma=1.5):
    coords = torch.arange(kernel_size) - kernel_size // 2
    g = torch.exp(-(coords**2) / (2 * sigma**2))
    g = g / g.sum()
    kernel = g[:,None,None] * g[None,:,None] * g[None,None,:]
    kernel = kernel[None,None,:,:,:]
    kernel = kernel.repeat(channels, 1, 1, 1, 1)
    return kernel

def masked_ssim_3d(pred, true, mask, C1=0.01**2, C2=0.03**2):
    """
    pred, true, mask: shape (1,1,H,W,D)
    """
    kernel = gaussian_kernel_3d(1).to(pred.device)

    mu_pred = F.conv3d(pred, kernel, padding=5)
    mu_true = F.conv3d(true, kernel, padding=5)

    mu_pred_sq = mu_pred ** 2
    mu_true_sq = mu_true ** 2
    mu_pred_true = mu_pred * mu_true

    sigma_pred = F.conv3d(pred * pred, kernel, padding=5) - mu_pred_sq
    sigma_true = F.conv3d(true * true, kernel, padding=5) - mu_true_sq
    sigma_pred_true = F.conv3d(pred * true, kernel, padding=5) - mu_pred_true

    ssim_num = (2 * mu_pred_true + C1) * (2 * sigma_pred_true + C2)
    ssim_den = (mu_pred_sq + mu_true_sq + C1) * (sigma_pred + sigma_true + C2)

    ssim = ssim_num / (ssim_den + 1e-6)

    # apply brain mask
    masked_ssim = ssim[mask > 0].mean()
    return masked_ssim


In [21]:
from tqdm import tqdm
import numpy as np
import scipy
def get_features(dataloader, encoder, device):
    feats = []

    encoder.eval()
    with torch.no_grad():
        for rest, task in tqdm(dataloader):
            task = task.to(device)
            z = encoder(task)[0]  # shape (B, latent_dim)
            feats.append(z.cpu().numpy())

    feats = np.concatenate(feats, axis=0)
    return feats


In [22]:
def compute_fid(feats1, feats2):
    mu1, mu2 = feats1.mean(0), feats2.mean(0)
    cov1 = np.cov(feats1, rowvar=False)
    cov2 = np.cov(feats2, rowvar=False)

    diff = mu1 - mu2
    covmean = scipy.linalg.sqrtm(cov1 @ cov2)

    fid = diff.dot(diff) + np.trace(cov1 + cov2 - 2 * covmean)
    return np.real(fid)


In [23]:
import matplotlib.pyplot as plt
import numpy as np
import os


In [24]:
def visualize_3d_slices(X, title, save_path):
    """
    X: numpy array (H,W,D)
    Plots axial, sagittal, and coronal slices.
    """
    H, W, D = X.shape
    slices = [
        X[:, :, D // 2],   # axial (z mid)
        X[:, W // 2, :],   # sagittal (y mid)
        X[H // 2, :, :],   # coronal (x mid)
    ]
    slice_titles = ["Axial (Z)", "Sagittal (Y)", "Coronal (X)"]

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    for i, sl in enumerate(slices):
        im = axs[i].imshow(sl.T, cmap="jet", origin="lower")
        axs[i].set_title(slice_titles[i])
        plt.colorbar(im, ax=axs[i], fraction=0.046)
    fig.suptitle(title)
    fig.tight_layout()
    fig.savefig(save_path)
    plt.close()


In [25]:
def plot_histogram_comparison(true_img, pred_img, save_path):
    """
    true_img, pred_img: numpy arrays (H,W,D)
    """
    true_vals = true_img.flatten()
    pred_vals = pred_img.flatten()
    diff_vals = pred_vals - true_vals

    plt.figure(figsize=(10, 5))
    
    plt.hist(true_vals, bins=80, alpha=0.5, label="True", density=True)
    plt.hist(pred_vals, bins=80, alpha=0.5, label="Predicted", density=True)
    
    plt.legend()
    plt.title("Histogram Comparison")
    plt.xlabel("Intensity")
    plt.ylabel("Density")
    plt.grid(alpha=0.3)
    plt.savefig(save_path.replace(".png", "_hist.png"))
    plt.close()

    # Difference histogram
    plt.figure(figsize=(10, 4))
    plt.hist(diff_vals, bins=80, alpha=0.7, color="purple")
    plt.title("Histogram of Prediction Error (Pred - True)")
    plt.xlabel("Difference")
    plt.ylabel("Count")
    plt.grid(alpha=0.3)
    plt.savefig(save_path.replace(".png", "_diffhist.png"))
    plt.close()


In [26]:
def visualize_intermediate_states(axials, titles, save_path):
    """
    axials: list of 2D axial slices (H,W)
    titles: list of strings
    """
    import matplotlib.pyplot as plt

    n = len(axials)
    plt.figure(figsize=(3*n, 4))

    for i in range(n):
        plt.subplot(1, n, i+1)
        plt.imshow(axials[i], cmap="gray")
        plt.title(titles[i])
        plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()


In [32]:
def test_and_save_predictions():

    # ----------------------------
    # 1. Paths (EDIT THESE)
    # ----------------------------
    rest_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/"
    task_dir = "/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/"
    mask_path = "/nfs/turbo/sph-jianghui1/yutingd/HCP/AAL_MNI_2mm.nii"

    
    test_ids  = load_ids("/nfs/turbo/sph-jianghui1/yutingd/HCP/test_ids.txt")

    
    save_dir = "SB_predictions"
    os.makedirs(save_dir, exist_ok=True)

    # ----------------------------
    # 2. Load mask
    # ----------------------------
    mask_np = nib.load(mask_path).get_fdata()
    mask_np = (mask_np > 0).astype(np.float32)
    mask_t  = torch.tensor(mask_np, dtype=torch.float32)[None, None]

    # ----------------------------
    # 3. Dataset
    # ----------------------------
    test_ds = HCPRestTaskDataset(rest_dir, task_dir, test_ids, mask_np=mask_np)
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # ----------------------------
    # 4. Load trained AE
    # ----------------------------
    latent_dim = 128
    # -------- Load trained AE --------
    encoder = Encoder3DStrong(latent_dim).to(device)

    # run 1 sample through encoder to get feature_shape
    rest, task = next(iter(test_loader))
    #X = torch.cat([rest, task], dim=0).to(device)
    X = task.to(device)


    with torch.no_grad():
        z, feature_shape = encoder(X)
    _, _, H0, W0, D0 = X.shape

    decoder = Decoder3DStrong(
        latent_dim,
        feature_shape=feature_shape,
        out_spatial_shape=(H0, W0, D0),
    ).to(device)
    encoder.load_state_dict(torch.load("encoder_ae_best.pth", map_location=device))
    decoder.load_state_dict(torch.load("decoder_ae_best.pth", map_location=device))

    encoder.eval()
    decoder.eval()

    for p in encoder.parameters():
        p.requires_grad = False
    for p in decoder.parameters():
        p.requires_grad = False

    # ----------------------------
    # 5. Load trained drift net
    # ----------------------------
    drift_net = DriftNet(latent_dim=latent_dim).to(device)
    drift_net.load_state_dict(torch.load("driftnet_sb_best.pth", map_location=device))
    drift_net.eval()

    # ----------------------------
    # 6. Loop test set
    # ----------------------------
    mse_list, corr_list = [], []
    ssim_list = []
    fid_feats_true = []
    fid_feats_pred = []


    print("[SB Test] Running predictions...")

    for (rest, task), sid in tqdm(zip(test_loader, test_ids), total=len(test_ids)):

        rest = rest.to(device)  # (1,1,H,W,D)
        task = task.to(device)
        H, W, D = rest.shape[2:5]

        # Encode rest → latent
        z_r, _ = encoder(rest)
        z_r_n = (z_r - z_mean) / z_std



        # Integrate latent ODE to get predicted latent task
        z_pred_n = euler_integrate(z_r_n, drift_net, steps=50)
        z_pred   = z_pred_n * z_std + z_mean
        
        # ---------------------------------------------
        # Compute intermediate latent states manually
        # ---------------------------------------------
        t_values = [0.0, 0.25, 0.5, 0.75, 1.0]

        z_states = []
        for t in t_values:
            t_tensor = torch.full((1,), t, device=device)
            # one Euler step: predict drift at (z_r_n + t*(z_pred_n - z_r_n))
            z_t = (1 - t) * z_r_n + t * z_pred_n
            z_t = z_t * z_std + z_mean  # unstandardize
            z_states.append(z_t)

        # ---------------------------------------------
        # Decode all 5 states
        # ---------------------------------------------
        decoded_states = []
        for z in z_states:
            X_t = decoder(z, rest.shape)
            X_t = (X_t * mask_t_dev).squeeze().detach().cpu().numpy()
            decoded_states.append(X_t)
            
        # axial slice visualization
        mid_z = decoded_states[0].shape[2] // 2
        axials = [x[:, :, mid_z] for x in decoded_states]
        titles = ["t=0", "t=0.25", "t=0.5", "t=0.75", "t=1"]

        interp_path = os.path.join(sub_dir, f"{sid}_interp.png")
        visualize_intermediate_states(axials, titles, interp_path)
        # -------------------------------------------------------



        # Decode to full fMRI volume
        X_pred = decoder(z_pred, rest.shape)
        mask_t_dev = mask_t.to(device)
        X_pred = X_pred * mask_t_dev   # zero-out outside brain
        
        X_true = task
        
        sub_dir = os.path.join(save_dir, "diagnostics", str(sid))
        os.makedirs(sub_dir, exist_ok=True)

        # Convert tensors to numpy arrays
        pred_np = X_pred.squeeze().detach().cpu().numpy()
        true_np = X_true.squeeze().detach().cpu().numpy()
        err_np  = pred_np - true_np

        # ---- 1. Histogram comparisons ----
        plot_histogram_comparison(true_np, pred_np,
                                  save_path=os.path.join(sub_dir, f"{sid}_hist.png"))
        # ---- 2. Brain slices: prediction ----
        visualize_3d_slices(
            pred_np,
            title=f"Predicted ({sid})",
            save_path=os.path.join(sub_dir, f"{sid}_pred.png")
        )

        # ---- 3. Brain slices: true image ----
        visualize_3d_slices(
            true_np,
            title=f"True Task ({sid})",
            save_path=os.path.join(sub_dir, f"{sid}_true.png")
        )

        # ---- 4. Error map visualization ----
        visualize_3d_slices(
            err_np,
            title=f"Error Map (Pred - True) {sid}",
            save_path=os.path.join(sub_dir, f"{sid}_error.png")
        )
                # Compute masked metrics
        mask_t_dev = mask_t.to(device)
        mse = masked_mse(X_pred, X_true, mask_t_dev).item()
        corr = masked_corr(X_pred, X_true, mask_t_dev).item()
        ssim = masked_ssim_3d(X_pred, task, mask_t_dev).item()

        mse_list.append(mse)
        corr_list.append(corr)
        ssim_list.append(ssim)
        # latent FID features
        with torch.no_grad():
            z_true = encoder(task)[0].detach().cpu().numpy()
            z_pred = encoder(X_pred)[0].detach().cpu().numpy()

        fid_feats_true.append(z_true)
        fid_feats_pred.append(z_pred)

        # ---------------------------------------------
        # 7. Save predicted NIfTI file
        # ---------------------------------------------
        pred_np = X_pred.squeeze().detach().cpu().numpy()  # (H,W,D)

        # Load reference affine (keep spatial location correct)
        ref_path = os.path.join(rest_dir, f"{sid}_fALFF.nii")
        aff = nib.load(ref_path).affine

        # Save predicted task image
        save_path = os.path.join(save_dir, f"{sid}_SBpred.nii")
        nib.save(nib.Nifti1Image(pred_np, affine=aff), save_path)


    print("---- Test Summary ----")
    print(f"Mean Masked MSE:  {np.mean(mse_list):.4f}")
    print(f"Mean Masked Corr: {np.mean(corr_list):.4f}")
    print(f"Mean Masked SSIM: {np.mean(ssim_list):.4f}")
    
    fid_val = compute_fid(np.vstack(fid_feats_true),
                      np.vstack(fid_feats_pred))
    
    print(f"FID: {np.mean(fid_val):.4f}")


    print(f"[SB Test] Saved predictions to {save_dir}/")


if __name__ == "__main__":
    test_and_save_predictions()

[Dataset] Loaded 73 subjects


/tmp/ipykernel_3148339/2345917193.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("encoder_ae_best.pth", map_location=device))
/tmp/i

[SB Test] Running predictions...


  0%|          | 0/73 [00:00<?, ?it/s]


NameError: name 'z_mean' is not defined

In [56]:
print("Task true  mean/std:", task.mean(dim=[1,2,3,4], keepdim=True), task.std().item())
#print("Task recon mean/std:", X_pred.mean().item(), X_pred.std().item())


Task true  mean/std: tensor([[[[[ 1.3524e-09]]]],



        [[[[ 2.9212e-08]]]],



        [[[[-1.9474e-08]]]],



        [[[[-5.6800e-09]]]],



        [[[[ 6.2210e-09]]]],



        [[[[ 1.7581e-08]]]],



        [[[[-3.2052e-08]]]],



        [[[[-7.8438e-09]]]],



        [[[[ 1.9610e-08]]]],



        [[[[-8.1143e-09]]]],



        [[[[ 2.1909e-08]]]],



        [[[[-1.7581e-08]]]],



        [[[[-3.0158e-08]]]],



        [[[[-1.7851e-08]]]],



        [[[[-7.0324e-09]]]],



        [[[[ 5.6800e-09]]]]], device='cuda:0') 1.0


In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

def plot_three_views(
    rest_nii,
    true_task_nii,
    pred_task_nii,
    save_path="sb_vis.png",
    title=None,
):
    """
    Plot axial, sagittal, coronal slices for:
        - rest
        - true task
        - predicted task

    Inputs are file paths to NIfTI files.
    """

    # --------------------------
    # Load volumes
    # --------------------------
    rest = nib.load(rest_nii).get_fdata()
    true = nib.load(true_task_nii).get_fdata()
    pred = nib.load(pred_task_nii).get_fdata()

    # Ensure float32
    rest = np.array(rest, dtype=np.float32)
    true = np.array(true, dtype=np.float32)
    pred = np.array(pred, dtype=np.float32)

    # --------------------------
    # Choose slices (middle slices)
    # --------------------------
    H, W, D = rest.shape

    slice_x = H // 2   # sagittal
    slice_y = W // 2   # coronal
    slice_z = D // 2   # axial

    # --------------------------
    # Three views × 3 images = 9 panels
    # --------------------------
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    plt.subplots_adjust(wspace=0.02, hspace=0.02)

    # Helper to remove borders
    def clean_axis(ax):
        ax.set_xticks([])
        ax.set_yticks([])

    # --------------------------
    # Row 1: AXIAL (z)
    # --------------------------
    axes[0, 0].imshow(rest[:, :, slice_z].T, cmap="gray", origin="lower")
    axes[0, 1].imshow(true[:, :, slice_z].T, cmap="gray", origin="lower")
    axes[0, 2].imshow(pred[:, :, slice_z].T, cmap="gray", origin="lower")
    axes[0, 0].set_ylabel("Axial", fontsize=12)

    # --------------------------
    # Row 2: SAGITTAL (x)
    # --------------------------
    axes[1, 0].imshow(rest[slice_x, :, :].T, cmap="gray", origin="lower")
    axes[1, 1].imshow(true[slice_x, :, :].T, cmap="gray", origin="lower")
    axes[1, 2].imshow(pred[slice_x, :, :].T, cmap="gray", origin="lower")
    axes[1, 0].set_ylabel("Sagittal", fontsize=12)

    # --------------------------
    # Row 3: CORONAL (y)
    # --------------------------
    axes[2, 0].imshow(rest[:, slice_y, :].T, cmap="gray", origin="lower")
    axes[2, 1].imshow(true[:, slice_y, :].T, cmap="gray", origin="lower")
    axes[2, 2].imshow(pred[:, slice_y, :].T, cmap="gray", origin="lower")
    axes[2, 0].set_ylabel("Coronal", fontsize=12)

    # --------------------------
    # Column Titles
    # --------------------------
    axes[0, 0].set_title("Rest", fontsize=14)
    axes[0, 1].set_title("True Task", fontsize=14)
    axes[0, 2].set_title("SB Predicted", fontsize=14)

    # Clean ticks
    for ax_row in axes:
        for ax in ax_row:
            clean_axis(ax)

    if title is not None:
        fig.suptitle(title, fontsize=16)

    # Save image
    fig.savefig(save_path, dpi=200, bbox_inches="tight")
    plt.close(fig)

    print(f"[Saved] {save_path}")


if __name__ == "__main__":

    # Example usage
    subject = "100307"

    rest_nii = f"/nfs/turbo/sph-jianghui1/yutingd/HCP/fALFF/{subject}_fALFF.nii"
    true_task_nii = f"/nfs/turbo/sph-jianghui1/yutingd/HCP/WM_Contrasts_2bk-0bk/{subject}_2bk-0bk.nii"
    pred_task_nii = f"SB_predictions/{subject}_SBpred.nii"

    plot_three_views(
        rest_nii, true_task_nii, pred_task_nii,
        save_path=f"sb_{subject}.png",
        title=f"Subject {subject}: Rest vs Task vs SB Prediction"
    )


[Saved] sb_100307.png


In [46]:
test_ids

['149842',
 '620434',
 '300618',
 '456346',
 '131924',
 '638049',
 '283543',
 '100307',
 '885975',
 '628248',
 '126426',
 '162329',
 '111009',
 '536647',
 '353740',
 '395958',
 '220721',
 '129129',
 '207426',
 '635245',
 '209935',
 '173940',
 '103414',
 '200008',
 '727553',
 '274542',
 '186141',
 '151627',
 '987074',
 '835657',
 '481042',
 '955465',
 '524135',
 '174841',
 '645450',
 '633847',
 '157942',
 '453441',
 '541943',
 '118730',
 '449753',
 '627852',
 '177645',
 '105923',
 '199453',
 '412528',
 '149539',
 '186848',
 '158136',
 '268749',
 '325129',
 '506234',
 '209228',
 '198249',
 '873968',
 '139435',
 '878776',
 '130720',
 '238033',
 '128127',
 '406836',
 '205119',
 '165638',
 '424939',
 '136833',
 '756055',
 '105620',
 '965771',
 '123723',
 '132017',
 '185442',
 '371843',
 '131217']